<!-- HTML file automatically generated from DocOnce source (https://github.com/doconce/doconce/)
doconce format html exercisesweek37.do.txt  -->
<!-- dom:TITLE: Exercises week 37 -->

# Exercises week 37
**Implementing gradient descent for Ridge and ordinary Least Squares Regression**

Date: **September 8-12, 2025**

## Learning goals

After having completed these exercises you will have:
1. Your own code for the implementation of the simplest gradient descent approach applied to ordinary least squares (OLS) and Ridge regression

2. Be able to compare the analytical expressions for OLS and Ridge regression with the gradient descent approach

3. Explore the role of the learning rate in the gradient descent approach and the hyperparameter $\lambda$ in Ridge regression

4. Scale the data properly

## Simple one-dimensional second-order polynomial

We start with a very simple function

$$
f(x)= 2-x+5x^2,
$$

defined for $x\in [-2,2]$. You can add noise if you wish. 

We are going to fit this function with a polynomial ansatz. The easiest thing is to set up a second-order polynomial and see if you can fit the above function.
Feel free to play around with higher-order polynomials.

## Exercise 1, scale your data

Before fitting a regression model, it is good practice to normalize or
standardize the features. This ensures all features are on a
comparable scale, which is especially important when using
regularization. Here we will perform standardization, scaling each
feature to have mean 0 and standard deviation 1.

### 1a)

Compute the mean and standard deviation of each column (feature) in your design/feature matrix $\boldsymbol{X}$.
Subtract the mean and divide by the standard deviation for each feature.

We will also center the target $\boldsymbol{y}$ to mean $0$. Centering $\boldsymbol{y}$
(and each feature) means the model does not require a separate intercept
term, the data is shifted such that the intercept is effectively 0
. (In practice, one could include an intercept in the model and not
penalize it, but here we simplify by centering.)
Choose $n=100$ data points and set up $\boldsymbol{x}$, $\boldsymbol{y}$ and the design matrix,    $\boldsymbol{X}$.

In [52]:
import numpy as np 

n = 10
x = np.linspace (-2,2, n)
y = 2 - x + 5*x**2

noise = np.random.normal(0,1,n)
y_noisy = y + 0.1 * noise

X = np.zeros((n,3)) 
print(X.shape)
for i in range(0,3):
    X[:,i] = x**(i+1)

print(X)

# Standardize features (zero mean, unit variance for each feature)
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_std[X_std == 0] = 1  # safeguard to avoid division by zero for constant features
X_norm = (X - X_mean) / X_std

# Center the target to zero mean (optional, to simplify intercept handling)
y_mean = np.mean(y, axis=0)   
y_centered = y - y_mean
print(X_norm)
print(y)
print(y_centered)

(10, 3)
[[-2.          4.         -8.        ]
 [-1.55555556  2.41975309 -3.76406036]
 [-1.11111111  1.2345679  -1.37174211]
 [-0.66666667  0.44444444 -0.2962963 ]
 [-0.22222222  0.04938272 -0.01097394]
 [ 0.22222222  0.04938272  0.01097394]
 [ 0.66666667  0.44444444  0.2962963 ]
 [ 1.11111111  1.2345679   1.37174211]
 [ 1.55555556  2.41975309  3.76406036]
 [ 2.          4.          8.        ]]
[[-1.5666989   1.65144565 -1.99828049]
 [-1.21854359  0.55048188 -0.94020604]
 [-0.87038828 -0.27524094 -0.34264069]
 [-0.52223297 -0.82572282 -0.07401039]
 [-0.17407766 -1.10096377 -0.00274113]
 [ 0.17407766 -1.10096377  0.00274113]
 [ 0.52223297 -0.82572282  0.07401039]
 [ 0.87038828 -0.27524094  0.34264069]
 [ 1.21854359  0.55048188  0.94020604]
 [ 1.5666989   1.65144565  1.99828049]]
[24.         15.65432099  9.28395062  4.88888889  2.4691358   2.02469136
  3.55555556  7.0617284  12.54320988 20.        ]
[13.85185185  5.50617284 -0.86419753 -5.25925926 -7.67901235 -8.12345679
 -6.59259259 -

Fill in the necessary details. Do we need to center the $y$-values? 
It looks like they get better centered around 0 but maybe not very big differnce in this case. 

After this preprocessing, each column of $\boldsymbol{X}_{\mathrm{norm}}$ has mean zero and standard deviation $1$
and $\boldsymbol{y}_{\mathrm{centered}}$ has mean 0. This makes the optimization landscape
nicer and ensures the regularization penalty $\lambda \sum_j
\theta_j^2$ in Ridge regression treats each coefficient fairly (since features are on the
same scale).

## Exercise 2, calculate the gradients

Find the gradients for OLS and Ridge regression using the mean-squared error as cost/loss function.

The derivative for OLS is simply done by taking the cost function and diferentiate w.r.t. $\theta$: $\frac{2}{n}X^T(X\theta - \mathbf{y})$ 

The derivative for Rigde is done the same way but here we have the additional parameter lamda: $2 (\frac{1}{n}X^T(X\theta - \mathbf{y})+\lambda \theta)$

## Exercise 3, using the analytical formulae for OLS and Ridge regression to find the optimal paramters $\boldsymbol{\theta}$

In [53]:
# Set regularization parameter, either a single value or a vector of values
# Note that lambda is a python keyword. The lambda keyword is used to create small, single-expression functions without a formal name. These are often called "anonymous functions" or "lambda functions."
lam = 0.1 #10 #0.001 #0.0001
n_features = 2

# Analytical form for OLS and Ridge solution: theta_Ridge = (X^T X + lambda * I)^{-1} X^T y and theta_OLS = (X^T X)^{-1} X^T y
I = np.eye(n_features)
def Ridge_parameters(X, y,lam):

    I = np.eye(X.shape[1])  # Identity matrix of size (n_features, n_features)
    
    # Ridge formula
    beta_ridge = np.linalg.inv(X.T @ X + lam * I) @ (X.T @ y)
    
    return beta_ridge



def OLS_parameters(X, y):
    beta_OLS = np.linalg.inv(X.T @ X) @ X.T @ y
    return beta_OLS


beta = OLS_parameters(X,y)

theta_closed_formRidge = Ridge_parameters(X,y,lam)
theta_closed_formOLS = OLS_parameters(X,y)

print("Closed-form Ridge coefficients:", theta_closed_formRidge)
print("Closed-form OLS coefficients:", theta_closed_formOLS)

Closed-form Ridge coefficients: [-0.96046205  5.67908378 -0.0116262 ]
Closed-form OLS coefficients: [-1.00000000e+00  5.69112628e+00  1.11022302e-15]


This computes the Ridge and OLS regression coefficients directly. The identity
matrix $I$ has the same size as $X^T X$. It adds $\lambda$ to the diagonal of $X^T X$ for Ridge regression. We
then invert this matrix and multiply by $X^T y$. The result
for $\boldsymbol{\theta}$  is a NumPy array of shape (n$\_$features,) containing the
fitted parameters $\boldsymbol{\theta}$.

### 3a)

Finalize, in the above code, the OLS and Ridge regression determination of the optimal parameters $\boldsymbol{\theta}$.

### 3b)

Explore the results as function of different values of the hyperparameter $\lambda$. See for example exercise 4 from week 36.

## Exercise 4, Implementing the simplest form for gradient descent

Alternatively, we can fit the ridge regression model using gradient
descent. This is useful to visualize the iterative convergence and is
necessary if $n$ and $p$ are so large that the closed-form might be
too slow or memory-intensive. We derive the gradients from the cost
functions defined above. Use the gradients of the Ridge and OLS cost functions with respect to
the parameters  $\boldsymbol{\theta}$ and set up (using the template below) your own gradient descent code for OLS and Ridge regression.

Below is a template code for gradient descent implementation of ridge:

In [54]:
# Gradient descent parameters, learning rate eta first
eta = 0.001
# Then number of iterations
num_iters = 100
# Initialize weights for gradient descent
theta = np.zeros(n_features)

n_samples, n_features = X.shape

theta_gdOLS = np.zeros(n_features)
theta_gdRidge = np.zeros(n_features)

print(y)
print(X)


def ols_gradient(X, y, theta):
    grad_o = (2/n) * X.T @ (X @ theta - y)
    return grad_o

def ridge_gradient(X, y, theta, lam):
    grad_r = 2 * ((1/n) * (X.T @ (X @ theta - y)) + lam * theta)
    return grad_r

# Gradient descent loop
for t in range(num_iters):
    # Compute gradients for OSL and Ridge
    grad_OLS = ols_gradient(X, y, theta_gdOLS)
    grad_Ridge = ridge_gradient(X,y,theta_gdRidge,lam)
    # Update parameters theta)
    theta_gdOLS = theta_gdOLS - eta * grad_OLS
    theta_gdRidge = theta_gdRidge - eta * grad_Ridge


# After the loop, theta contains the fitted coefficients
#theta_gdOLS = ?
#theta_gdRidge = ?
print("Gradient Descent OLS coefficients:", theta_gdOLS)
print("Gradient Descent Ridge coefficients:", theta_gdRidge)

[24.         15.65432099  9.28395062  4.88888889  2.4691358   2.02469136
  3.55555556  7.0617284  12.54320988 20.        ]
[[-2.          4.         -8.        ]
 [-1.55555556  2.41975309 -3.76406036]
 [-1.11111111  1.2345679  -1.37174211]
 [-0.66666667  0.44444444 -0.2962963 ]
 [-0.22222222  0.04938272 -0.01097394]
 [ 0.22222222  0.04938272  0.01097394]
 [ 0.66666667  0.44444444  0.2962963 ]
 [ 1.11111111  1.2345679   1.37174211]
 [ 1.55555556  2.41975309  3.76406036]
 [ 2.          4.          8.        ]]
Gradient Descent OLS coefficients: [-0.11937933  3.48495441 -0.25417965]
Gradient Descent Ridge coefficients: [-0.11858399  3.45577595 -0.25293918]


### 4a)

Write first a gradient descent code for OLS only using the above template.
Discuss the results as function of the learning rate parameters and the number of iterations

### 4b)

Write then a similar code for Ridge regression using the above template.
Try to add a stopping parameter as function of the number iterations and the difference between the new and old $\theta$ values. How would you define a stopping criterion?

In [55]:
tol = 1e-6
for t in range(num_iters):
    grad_OLS = ols_gradient(X, y, theta_gdOLS)
    new_theta = theta_gdOLS - eta * grad_OLS
    
    if np.linalg.norm(new_theta - theta_gdOLS) < tol:
        print(f"Converged after {t} iterations")
        break
    
    theta_gdOLS = new_theta

## Exercise 5, Ridge regression and a new Synthetic Dataset

We create a synthetic linear regression dataset with a sparse
underlying relationship. This means we have many features but only a
few of them actually contribute to the target. In our example, we’ll
use 10 features with only 3 non-zero weights in the true model. This
way, the target is generated as a linear combination of a few features
(with known coefficients) plus some random noise. The steps we include are:

Decide on the number of samples and features (e.g. 100 samples, 10 features).
Define the **true** coefficient vector with mostly zeros (for sparsity). For example, we set $\hat{\boldsymbol{\theta}} = [5.0, -3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0]$, meaning only features 0, 1, and 6 have a real effect on y.

Then we sample feature values for $\boldsymbol{X}$ randomly (e.g. from a normal distribution). We use a normal distribution so features are roughly centered around 0.
Then we compute the target values $y$ using the linear combination $\boldsymbol{X}\hat{\boldsymbol{\theta}}$ and add some noise (to simulate measurement error or unexplained variance).

Below is the code to generate the dataset:

In [61]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(0)

# Define dataset size
n_samples = 100
n_features = 10

# Define true coefficients (sparse linear relationship)
theta_true = np.array([5.0, -3.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0])

# Generate feature matrix X (n_samples x n_features) with random values
X = np.random.randn(n_samples, n_features)  # standard normal distribution

# Generate target values y with a linear combination of X and theta_true, plus noise
noise = 0.5 * np.random.randn(n_samples)    # Gaussian noise
y = X.dot(theta_true) + noise

def ols_closed_form(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

def ridge_closed_form(X, y, lam):
    I = np.eye(X.shape[1])
    return np.linalg.inv(X.T @ X + lam * I) @ X.T @ y

theta_ols_closed = ols_closed_form(X, y)
theta_ridge_closed = ridge_closed_form(X, y, lam=1.0)

def ols_gradient(X, y, theta):
    n = X.shape[0]
    return (2/n) * X.T @ (X @ theta - y)

def ridge_gradient(X, y, theta, lam):
    n = X.shape[0]
    return 2 * ((1/n) * (X.T @ (X @ theta - y)) + lam * theta)

theta_ols_gd = gradient_descent(X, y, ols_gradient, eta=0.01)
theta_ridge_gd = gradient_descent(X, y, ridge_gradient, eta=0.01, lam=1.0)


def gradient_descent(X, y, grad_fn, eta=0.01, lam=0.0, 
                     num_iters=10000, tol=1e-6):
    theta = np.zeros(X.shape[1])
    for t in range(num_iters):
        grad = grad_fn(X, y, theta) if lam == 0 else grad_fn(X, y, theta, lam)
        new_theta = theta - eta * grad
        if np.linalg.norm(new_theta - theta) < tol:
            break
        theta = new_theta
    return theta

print("True coefficients:       ", np.round(theta_true, 2))
print("OLS closed form:         ", np.round(theta_ols_closed, 2))
print("OLS gradient descent:    ", np.round(theta_ols_gd, 2))
print("Ridge closed form (λ=1): ", np.round(theta_ridge_closed, 2))
print("Ridge gradient descent:  ", np.round(theta_ridge_gd, 2))



True coefficients:        [ 5. -3.  0.  0.  0.  0.  2.  0.  0.  0.]
OLS closed form:          [ 5.01 -3.   -0.02  0.14 -0.07 -0.04  2.06  0.    0.04 -0.05]
OLS gradient descent:     [ 5.01 -3.   -0.02  0.14 -0.07 -0.04  2.06  0.    0.04 -0.05]
Ridge closed form (λ=1):  [ 4.96 -2.96 -0.02  0.15 -0.08 -0.05  2.03  0.    0.03 -0.05]
Ridge gradient descent:   [ 2.56 -1.3  -0.03  0.23 -0.23 -0.21  0.91  0.09 -0.21 -0.06]


This code produces a dataset where only features 0, 1, and 6
significantly influence $\boldsymbol{y}$. The rest of the features have zero true
coefficient. For example, feature 0 has
a true weight of 5.0, feature 1 has -3.0, and feature 6 has 2.0, so
the expected relationship is:

$$
y \approx 5 \times x_0 \;-\; 3 \times x_1 \;+\; 2 \times x_6 \;+\; \text{noise}.
$$

You can remove the noise if you wish to. 

Try to fit the above data set using OLS and Ridge regression with the analytical expressions and your own gradient descent codes.

If everything worked correctly, the learned coefficients should be
close to the true values [5.0, -3.0, 0.0, …, 2.0, …] that we used to
generate the data. Keep in mind that due to regularization and noise,
the learned values will not exactly equal the true ones, but they
should be in the same ballpark.  Which method (OLS or Ridge) gives the best results?